<a href="https://colab.research.google.com/github/amaneth/NLP_Week-1/blob/main/n_gram_models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


<h1 style="font-family:verdana;font-size:300%;text-align:center;background-color:#f2f2f2;color:#0d0d0d">AMMI NLP - Review sessions</h1>

<h1 style="font-family:verdana;font-size:180%;text-align:Center;color:#993333"> Lab 3: n-gram models </h1>

**Big thanks to Amr Khalifa who improved this lab and made it to a Jupyter Notebook!**

In [7]:
import io, sys, math, re
from collections import defaultdict
import numpy as np

In [8]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [9]:
# data_loader
def load_data(filename):
    '''
    parameters:
    filename (string): datafile
    
    Returns:
    data (list of lists): each list is a sentence of the text 
    vocab (dictionary): {word: no of times it appears in the text}
    '''
    fin = io.open(filename, 'r', encoding='utf-8')
    data = []
    vocab = defaultdict(lambda:0)
    for line in fin:
        sentence = line.split()
        data.append(sentence)
        for word in sentence:
            vocab[word] += 1
    return data, vocab

In [10]:
cd /content/drive/MyDrive/NLP_Week_1_Labs_2022/session3

/content/drive/.shortcut-targets-by-id/1hemX0olabA89fdb9njWGoX6q2g_RJU4Y/NLP_Week_1_Labs_2022/session3


In [11]:

print("load training set..")
print("\n")
train_data, vocab = load_data("train1.txt")
print(train_data[0])
print("\n")
print("how :",vocab['how'])
print("load validation set")
valid_data, _ = load_data("valid1.txt")


load training set..


['<s>', 'my', 'fathers', "don't", 'speak', 'dutch.', '</s>']


how : 107
load validation set


In [12]:
def remove_rare_words(data, vocab, mincount = 1):
    '''
    Parameters:
    data (list of lists): each list is a sentence of the text 
    vocab (dictionary): {word: no of times it appears in the text}
    mincount(int): the minimum count 
    
    Returns: 
    data_with_unk(list of lists): data after replacing rare words with <unk> token
    '''
    # replace words in data that are not in the vocab 
    # or have a count that is below mincount
    data_with_unk = []

    ## FILL CODE
    data_with_unk= [[word if vocab[word]>mincount else '<unk>' for word in sentence ] for sentence in data ]

    
    return data_with_unk

In [13]:
print("remove rare words")
train_data = remove_rare_words(train_data, vocab, mincount = 1)
valid_data = remove_rare_words(valid_data, vocab, mincount = 1)
#train_data
print(train_data[0])


remove rare words
['<s>', 'my', '<unk>', "don't", 'speak', '<unk>', '</s>']


In [14]:
def build_ngram(data, n):
    '''
    Parameters:
    data (list of lists): each list is a sentence of the text 
    n (int): size of the n-gram
    
    Returns:
    prob (dictionary of dictionary)
    {
        context: {word:probability of this word given context}
    }
    '''
    total_number_words = 0
    counts = defaultdict(lambda: defaultdict(lambda: 0.0))

    for sentence in data:
        sentence = tuple(sentence)
        ## FILL CODE
        # dict can be indexed by tuples
        # store in the same dict all the ngrams
        # by using the context as a key and the word as a value
        for gram in range(n):
          for i,word in enumerate(sentence[gram:]):
            counts[sentence[i:i+(gram)]][word]+=1
                           

    prob = defaultdict(lambda: defaultdict(lambda: 0.0))
    # Build the probabilities from the counts
    # Be careful with how you normalize!

    for context in counts.keys():
      for word,count in counts[context].items():
        prob[context][word]= counts[context][word]/sum(counts[context].values())
    ## FILL CODE


    return prob

In [111]:
# RUN TO BUILD NGRAM MODEL

n = 3
print("build ngram model with n = ", n)
model = build_ngram(train_data, n)

build ngram model with n =  3


Here, implement a recursive function over shorter and shorter context to compute a "stupid backoff model". An interpolation model can also be implemented this way.

In [123]:
def get_prob(model, context, w):
    '''
    Parameters: 
    model (dictionary of dictionary)
    {
        context: {word:probability of this word given context}
    } 
    context (list of strings): a sentence
    w(string): the word we need to find it's probability given the context
    
    Retunrs:
    prob(float): probability of this word given the context 
    '''

    # code a recursive function over 
    # smaller and smaller context
    # to compute the backoff model
    prob=0.0
    p=0.0
    ## FILL CODE
    def stupid(context,w):
      prefix = tuple(context)
      # print(prefix)
      # print(context)
      if model[prefix][w]>0:
        p=model[prefix][w]
        # print(p)
      else:
        context=context[1:]
        # print(context)
        p=0.4 * stupid(context,w)
      return p
    prob=stupid(context,w)

    return prob


In [124]:
def perplexity(model, data, n):
    '''
    Parameters: 
    model (dictionary of dictionary)
    {
        context: {word:probability of this word given context}
    } 
    data (list of lists): each list is a sentence of the text
    n(int): size of the n-gram
    
    Retunrs:
    perp(float): the perplexity of the model 
    '''
    probs=1
    N=0
    ## FILL CODE
    for doc in data:
      probs=probs*get_prob(model, doc[:-1], doc[-1])
      N+=len(doc)
      # print('perp',probs)
    preplexity=pow(probs,-(1/N))

    return preplexity

In [125]:
# COMPUTE PERPLEXITY ON VALIDATION SET

print("The perplexity is", perplexity(model, valid_data, n=n))

The perplexity is 2.1151567017235378


In [234]:
def get_proba_distrib(mod, cont):
    ## need to get the the words after the context and their probability of appearance
    ## after this context 
    '''
    Parameters: 
    model (dictionary of dictionary)
    {
        context: {word:probability of this word given context}
    }
    context (list of strings): the sentence we need to find the words after it and 
    thier probabilites
    
    Retunrs:
    words_and_probs(dic): {word: probability of word given context}
    
    '''
    # code a recursive function over context
    # to find the longest available ngram
    
    def prob(model, context):
      probs=None
      print('conntext', tuple(context))
      if tuple(context) in model.keys():
          probs=model[tuple(context)]
          print('return', probs)
          return probs
          
      else:
          context=context[1:]
          print('baccked off context', context )
          prob(model, context)

    answer=prob(mod, cont)
    print('why',answer)
    ## FILL CODE
    
    return answer

In [235]:
def generate(model):
    '''
    Parameters: 
    model (dictionary of dictionary)
    {
        context: {word:probability of this word given context}
    }
    
    Retunrs:
    sentence (list of strings): a sentence sampled according to the language model. 
    

    '''
    # generate a sentence. A sentence starts with a <s> and ends with a </s>
    # Possiblly a use function is:
    # np.random.choice(x, 1, p = y)

    # where x is a list of things to sample from
    # and y is a list of probability (of the same length as x)
    sentence = ["<s>"]
    while sentence[-1] != "</s>" and len(sentence)<100:
        ## FILL CODE
        prob_diztribution = get_proba_distrib(model, sentence)
        print('dist',prob_diztribution)
        choice = np.random.choice(a=list(prob_diztribution.keys()), p=list(prob_diztribution.values()))
        print('choice', choice)
        sentence.append(choice)


        
    return sentence

In [222]:
s = ["<s>"]
tuple(s) in model.keys()
sum(model[tuple(s)].values())

1.0000000000000044

In [186]:
model[tuple]

defaultdict(<function __main__.build_ngram.<locals>.<lambda>.<locals>.<lambda>>,
            {})

In [236]:
# GENERATE A SENTENCE FROM THE MODEL

print("Generated sentence: ",generate(model))

conntext ('<s>',)
return defaultdict(<function build_ngram.<locals>.<lambda>.<locals>.<lambda> at 0x7f4ac7a51dd0>, {'my': 0.013127135407300846, 'tomorrow': 0.0003596475454055026, 'he': 0.035425283222442004, 'tom': 0.2064376910627585, 'john': 0.000539471318108254, 'some': 0.0017982377270275131, 'when': 0.007552598453515555, "don't": 0.006114008271893544, 'that': 0.004315770544866032, 'sami': 0.03272792663190074, 'this': 0.010070131271354072, 'to': 0.002337709045135767, 'what': 0.011868368998381586, 'you': 0.014206078043517353, 'i': 0.1251573458011149, 'the': 0.058802373673799674, '<unk>': 0.03416651681352275, 'they': 0.051789246538392376, 'we': 0.01906131990649164, 'a': 0.005394713181082539, 'burn': 0.0003596475454055026, 'would': 0.0016184139543247618, 'in': 0.006653479590001799, 'being': 0.0001798237727027513, 'is': 0.00827189354432656, 'due': 0.000539471318108254, "i've": 0.0032368279086495235, 'where': 0.00557453695378529, 'if': 0.009710483725948571, 'there': 0.008092069771623809, "

AttributeError: ignored

Once you are done implementing the model, evaluation and generation code, you can try changing the value of `n`, and play with a larger training set (`train2.txt` and `valid2.txt`). You can also try to implement an interpolation model.